In [1]:
%load_ext autoreload
%autoreload 2

from jupyter_cadquery import set_defaults, show, Camera, Collapse
from build123d import *
from collections import namedtuple
from itertools import chain, product
from projects.tablet.features import Port, Button, CameraBump

set_defaults(
	viewer="Sidecar",
	edge_accuracy=0.0001,
	ticks=25,
	grid=[True, True, True],
	cad_width=780,
	axes=True,
	axes0=True,
	glass=True,
	orbit_control=True,
	reset_camera=Camera.CENTER,
	collapse=Collapse.LEAVES
)

Overwriting auto display for build123d BuildPart, BuildSketch, BuildLine, ShapeList


In [2]:
TABLET_LENGTH = 240
TABLET_WIDTH = 158
TABLET_HEIGHT = 10
TABLET_ROUND_RADIUS = 10
TABLET_CHAMFER = 2

CASE_THICKNESS = 1

In [3]:
class RawTablet(BasePartObject):
    def __init__(
        self,
        tablet_length,
        tablet_width,
        tablet_height,
        tablet_body_radius,
        tablet_chamfer,
        rotation = (0,0,0),
        align=(Align.CENTER, Align.CENTER, Align.MIN),
        mode=Mode.ADD
    ):
        with BuildPart() as tablet:
            Box(tablet_length, tablet_width, tablet_height)
            fillet(tablet.edges().filter_by(Axis.Z), tablet_body_radius)
            chamfer(tablet.faces().filter_by(Axis.Z).edges(), tablet_chamfer)
        super().__init__(tablet.part, rotation, align, mode)
        self.side_faces = tablet.part.faces().filter_by(GeomType.PLANE).sort_by(Axis.Z)[5:9]
        self.back_face = tablet.part.faces().sort_by(Axis.Z).first

with BuildPart() as tablet:
    t = RawTablet(TABLET_LENGTH, TABLET_WIDTH, TABLET_HEIGHT, TABLET_ROUND_RADIUS, TABLET_CHAMFER)
    Button(
        t.side_faces[0],
        (2, 10, 3),
        (0, 0),
        align=(Align.CENTER, Align.MAX, Align.CENTER)
    )
    Button(
        t.side_faces[2],
        (2, 20, 3),
        (0, 0),
        align=(Align.CENTER, Align.MIN, Align.CENTER)
    )
    Port(
        t.side_faces[3],
        (4, 10, 8), 
        (0,0),
        align=(Align.CENTER, Align.CENTER, Align.MAX)
    )
    CameraBump(
        t.back_face,
        6,
        1,
        (10,10),
        align=(Align.MIN, Align.MIN, Align.MIN)
    )

+


In [4]:
with BuildPart() as tablet_case:
    t = RawTablet(TABLET_LENGTH, TABLET_WIDTH, TABLET_HEIGHT, TABLET_ROUND_RADIUS, TABLET_CHAMFER)
    offset(openings=[t.faces().sort_by(Axis.Z).filter_by(Axis.Z).last], amount=CASE_THICKNESS)
    
    with BuildPart(tablet.faces().filter_by(Axis.Z).sort_by(Axis.Z).first, mode=Mode.SUBTRACT):
        Cylinder(8, 6, align=(Align.CENTER, Align.CENTER, Align.CENTER))

    with BuildPart(tablet.faces().filter_by(Axis.Y).sort_by(Axis.Y).last, mode=Mode.SUBTRACT):
        Box(8, 22, 10, align=(Align.CENTER, Align.CENTER, Align.CENTER))

    with BuildPart(tablet.faces().filter_by(Axis.X).sort_by(Axis.X).first, mode=Mode.SUBTRACT):
        Box(8, 12, 10, align=(Align.CENTER, Align.CENTER, Align.CENTER))

    with BuildPart(tablet.faces().filter_by(Axis.X).sort_by(Axis.X).last, mode=Mode.SUBTRACT):
        Box(8, 12, 10, align=(Align.CENTER, Align.CENTER, Align.CENTER))
    #split(bisect_by=Plane.XZ, keep=Keep.BOTTOM)
#show(tablet.faces().sort_by(Axis.Z).filter_by(Axis.Z).last)
show(tablet, tablet_case, colors=["grey", "green"])

++
